In [2]:
"""Create map figures for manuscript."""

import os

import pandas as pd
import numpy as np
import geopandas

import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as colors
import seaborn as sns
import xycmap

# Create bivariate choropleth map of wildfire and non-wildfire PM2.5 for the CONUS

In [3]:
if os.path.exists("temp/out_shape.pkl"):
    out_shape = geopandas.GeoDataFrame(pd.read_pickle("temp/out_shape.pkl"))

In [4]:
# use seaborn to scale up font
sns.set_context("paper", font_scale=1)

test = False

if test:
    start_year = 2007
    end_year = 2008
else:
    start_year = 2007
    end_year = 2018  # switch back to 2018

# set colormap shared across figures
cmap_crest = sns.color_palette("crest", as_cmap=True)

In [5]:
# get max and minimum values for colorbar

# get max and minimum values for nofire PM data
nofire_max = (
    out_shape[
        [
            "PM_nofire_2007",
            "PM_nofire_2008",
            "PM_nofire_2009",
            "PM_nofire_2010",
            "PM_nofire_2011",
            "PM_nofire_2012",
            "PM_nofire_2013",
            "PM_nofire_2014",
            "PM_nofire_2015",
            "PM_nofire_2016",
            "PM_nofire_2017",
            "PM_nofire_2018",
        ]
    ]
    .max()
    .max()
)

nofire_min = (
    out_shape[
        [
            "PM_nofire_2007",
            "PM_nofire_2008",
            "PM_nofire_2009",
            "PM_nofire_2010",
            "PM_nofire_2011",
            "PM_nofire_2012",
            "PM_nofire_2013",
            "PM_nofire_2014",
            "PM_nofire_2015",
            "PM_nofire_2016",
            "PM_nofire_2017",
            "PM_nofire_2018",
        ]
    ]
    .min()
    .min()
)

# get max and minimum values for wildfire PM data
wf_max = (
    out_shape[
        [
            "PM_wf_2007",
            "PM_wf_2008",
            "PM_wf_2009",
            "PM_wf_2010",
            "PM_wf_2011",
            "PM_wf_2012",
            "PM_wf_2013",
            "PM_wf_2014",
            "PM_wf_2015",
            "PM_wf_2016",
            "PM_wf_2017",
            "PM_wf_2018",
        ]
    ]
    .max()
    .max()
)

wf_min = (
    out_shape[
        [
            "PM_wf_2007",
            "PM_wf_2008",
            "PM_wf_2009",
            "PM_wf_2010",
            "PM_wf_2011",
            "PM_wf_2012",
            "PM_wf_2013",
            "PM_wf_2014",
            "PM_wf_2015",
            "PM_wf_2016",
            "PM_wf_2017",
            "PM_wf_2018",
        ]
    ]
    .min()
    .min()
)

In [ ]:
# plot wildfire PM2.5 for each year

# normalize the colorbar across all subplots
norm = colors.Normalize(vmin=0, vmax=wf_max)

fig, axs = plt.subplots(4, 3, figsize=(7, 7))
axs = axs.flatten()

for i, year in enumerate(range(start_year, end_year + 1)):
    # create figure
    out_shape.plot(
        f"PM_wf_{year}",
        ax=axs[i],
        cmap=cmap_crest,
        legend=False,
        antialiased=False,
        norm=norm,
    )

    axs[i].set_axis_off()

    # set figure title to year
    axs[i].set_title(f"{year}")

# add colorbar to bottom of subplots
sm = plt.cm.ScalarMappable(cmap=cmap_crest, norm=norm)
sm.set_array([])
fig.colorbar(
    sm,
    ax=axs,
    orientation="horizontal",
    pad=0.05,
    label="Annual mean fire PM$_{2.5}$ (μg/m³)",
)

fig.savefig(
    f"figures/PM25_wildland_fire_2007-2018.png",
    bbox_inches="tight",
    dpi=300,
)

In [ ]:
# plot nofire PM2.5 for each year

# normalize the colorbar across all subplots
norm = colors.Normalize(vmin=0, vmax=nofire_max)

fig, axs = plt.subplots(4, 3, figsize=(7, 7))
axs = axs.flatten()

for i, year in enumerate(range(start_year, end_year + 1)):
    # create figure
    out_shape.plot(
        f"PM_nofire_{year}",
        ax=axs[i],
        cmap=cmap_crest,
        legend=False,
        antialiased=False,
        norm=norm,
    )

    axs[i].set_axis_off()

    # set figure title to year
    axs[i].set_title(f"{year}")

# add colorbar to bottom of subplots
sm = plt.cm.ScalarMappable(cmap=cmap_crest, norm=norm)
sm.set_array([])
fig.colorbar(
    sm,
    ax=axs,
    orientation="horizontal",
    pad=0.05,
    label="Annual mean non-fire PM$_{2.5}$ (μg/m³)",
)

fig.savefig(
    f"figures/PM25_no_fire_2007-2018.png",
    bbox_inches="tight",
    dpi=300,
)

# plot where non-fire pm is less than threshold, and non-fire+wildfire pm is greater than threshold

In [ ]:
# create matplotlib colormap from list of hex codes from colorbrewer2.org
# original list from colorbrewer: "#a6cee3", "#1f78b4", "#b2df8a"
cmap2 = LinearSegmentedColormap.from_list(
    name="custom",
    colors=["#1f78b4", "lightgray", "#a6cee3"],
)


# set PM2.5 threshold to be plotted
# thresholds = [8, 9, 10]
threshold = 9.6

fig, axs = plt.subplots(4, 3, figsize=(7, 6))
axs = axs.flatten()

for i, year in enumerate(range(start_year, end_year + 1)):
    # create field indicating where nonfire pm2.5 is less than 9 but, nonfire+wf is greater than 9
    # and where nonfire pm2.5 is greater than 9
    out_shape[f"{year} greater than {threshold}"] = f"Total < {threshold}"
    out_shape.loc[
        out_shape[f"PM_nofire_{year}"] >= threshold,
        f"{year} greater than {threshold}",
    ] = f"Non-fire > {threshold}"

    out_shape.loc[
        (out_shape[f"PM_nofire_{year}"] < threshold)
        & (out_shape[f"PM_total_{year}"] >= threshold),
        f"{year} greater than {threshold}",
    ] = f"Total > {threshold}"

    out_shape = out_shape.sort_values(f"{year} greater than {threshold}")

    # create figure
    fig2 = out_shape.plot(
        f"{year} greater than {threshold}",
        ax=axs[i],
        cmap=cmap2,
        legend=True,
        categorical=True,
        antialiased=False,
    )
    legend = fig2.get_legend()
    # deactivate legend
    legend.set_visible(False)

    axs[i].set_axis_off()

    # set figure title to year
    axs[i].set_title(f"{year}")

handles = legend.legend_handles
labels = [v.get_text() for v in legend.texts]

# reorder legend items
handles[1], handles[2] = handles[2], handles[1]
labels[1], labels[2] = labels[2], labels[1]

fig.legend(
    handles=handles,
    labels=labels,
    loc="lower center",
    ncol=3,
    bbox_to_anchor=(0.5, -0.07),
    title="Annual mean PM$_{2.5}$ (μg/m³)",
)

fig.tight_layout()

fig.savefig(
    f"figures/PM25_exceeding_{threshold}_2007-2018.png",
    bbox_inches="tight",
    dpi=300,
)

In [ ]:
# Plot number of census tracts exceeding the threshold in each year
sns.set_theme("notebook", "ticks")
fig, ax = plt.subplots(1, 1, figsize=(8, 4.5), sharex=True, sharey=True)

out_shape_counts = (
    out_shape.loc[:, ["Total Population"] + out_shape.columns[80:].tolist()]
    .reset_index()
    .set_index(["GEOID", "Total Population"])
    .stack()
    .reset_index()
)

out_shape_counts.columns = ["GEOID", "Total Population", "Year", "exceedance"]
out_shape_counts["Year"] = out_shape_counts.Year.str.replace(
    " greater than 9.6", ""
).astype(int)

df_p = (
    out_shape_counts.groupby(["Year", "exceedance"])["Total Population"]
    .sum()
    .unstack()[["Non-fire > 9.6", "Total > 9.6", "Total < 9.6"]]
)

df_p = df_p / 1e6

df_p.plot(kind="bar", ax=ax, stacked=True, color=["#1f78b4", "#a6cee3", "lightgray"])

ax.tick_params(labelrotation=45)
ax.set_xlabel("")
ax.set_ylabel("Population (millions)")
sns.despine()

ax.legend(
    bbox_to_anchor=(0.5, -0.16),
    loc="upper center",
    ncol=3,
    frameon=True,
    fontsize=12,
    title="Annual mean PM$_{2.5}$ (μg/m³)",
)

# Plot total pm2.5 where both wildfire and non-fire pm2.5 are in top quartile

In [ ]:
# get max and minimum values for colorbar

overall_max = (
    out_shape[
        [
            "PM_total_2007",
            "PM_total_2008",
            "PM_total_2009",
            "PM_total_2010",
            "PM_total_2011",
            "PM_total_2012",
            "PM_total_2013",
            "PM_total_2014",
            "PM_total_2015",
            "PM_total_2016",
            "PM_total_2017",
            "PM_total_2018",
        ]
    ]
    .max()
    .max()
)

overall_min = (
    out_shape[
        [
            "PM_total_2007",
            "PM_total_2008",
            "PM_total_2009",
            "PM_total_2010",
            "PM_total_2011",
            "PM_total_2012",
            "PM_total_2013",
            "PM_total_2014",
            "PM_total_2015",
            "PM_total_2016",
            "PM_total_2017",
            "PM_total_2018",
        ]
    ]
    .min()
    .min()
)

# normalize the colorbar across all subplots
norm = colors.Normalize(vmin=5, vmax=overall_max)

fig, axs = plt.subplots(4, 3, figsize=(7, 7))
axs = axs.flatten()

for i, year in enumerate(range(start_year, end_year + 1)):
    # create field indicating where both wildfire and non-fire pm2.5 are in top 50%
    out_shape[f"{year} above medians"] = np.nan

    out_shape.loc[
        (out_shape[f"PM_nofire_{year}"].rank(pct=True) >= 0.5)
        & (out_shape[f"PM_wf_{year}"].rank(pct=True) >= 0.5),
        f"{year} above medians",
    ] = out_shape[f"PM_total_{year}"]

    # create figure
    out_shape.plot(
        f"{year} above medians",
        ax=axs[i],
        cmap=cmap_crest,
        legend=False,
        categorical=False,
        norm=norm,
        missing_kwds=dict(
            color="lightgrey",
        ),
        # antialiased=False,
    )
    axs[i].set_axis_off()

    # set figure title to year
    axs[i].set_title(f"{year}")

# add colorbar to bottom of subplots
sm = plt.cm.ScalarMappable(cmap=cmap_crest, norm=norm)
sm.set_array([])
fig.colorbar(
    sm,
    ax=axs,
    orientation="horizontal",
    pad=0.05,
    label="Annual mean PM$_{2.5}$ (μg/m³)",
)

fig.savefig(
    f"figures/PM25_in_top_50th_with_conc.png",
    bbox_inches="tight",
    dpi=300,
)

In [ ]:
# map total pm2.5 for all years

# normalize the colorbar across all subplots
norm = colors.Normalize(vmin=5, vmax=overall_max)

fig, axs = plt.subplots(4, 3, figsize=(7, 7))
axs = axs.flatten()

for i, year in enumerate(range(start_year, end_year + 1)):
    # create figure
    out_shape.plot(
        f"PM_total_{year}",
        ax=axs[i],
        cmap=cmap_crest,
        legend=False,
        norm=norm,
        antialiased=False,
    )
    axs[i].set_axis_off()

    # set figure title to year
    axs[i].set_title(f"{year}")

# add colorbar to bottom of subplots
sm = plt.cm.ScalarMappable(cmap=cmap_crest, norm=norm)
sm.set_array([])
fig.colorbar(
    sm,
    ax=axs,
    orientation="horizontal",
    pad=0.05,
    label="Annual mean PM$_{2.5}$ (μg/m³)",
)

fig.savefig(
    f"figures/PM25_total_2007-2018.png",
    bbox_inches="tight",
    dpi=300,
)